<a href="https://colab.research.google.com/github/20911357Pinyaphat/20911357-smart-finance-assistant/blob/main/diary/%20week8.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting Stock Data with APIs
**Learn to fetch real stock prices and create simple visualizations**

## What You'll Build
A simple tool that gets stock price data from a free API and creates a basic chart using pandas plotting.

## Setup

In [1]:
# Install what we need
!pip install pandas requests matplotlib

In [3]:
# Import libraries
import pandas as pd
import requests
import matplotlib.pyplot as plt
from datetime import datetime

## Step 1: Ask AI to Help Choose a Stock API

Let's start by asking Colab's AI about free stock APIs. But for this tutorial, we'll simulate the conversation.
```
Ask Colab AI: 'What are some free stock APIs I can use for learning? I want to get basic stock price data.'

AI Response would suggest options like:
- Alpha Vantage (free tier)
- Yahoo Finance (via yfinance)
- IEX Cloud (free tier)
- Finnhub (free tier)
```
For this tutorial, we'll use Alpha Vantage since it's beginner-friendly.

## Step 2: Ask AI to Help Build the API Call

Let's ask AI to help us structure the API request, you might get something like

In [15]:
def get_stock_data(symbol, api_key="demo"):
    """
    Get stock data from Alpha Vantage
    Using 'demo' key for testing (limited data)
    """

    base_url = "https://www.alphavantage.co/query"

    # Parameters for the API call
    params = {
        'function': 'TIME_SERIES_DAILY',
        'symbol': symbol,
        'apikey': api_key,
        'outputsize': 'compact'  # Last 100 days
    }

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status() # Raise an exception for bad status codes
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        print(f"Error during API call: {e}")
        return {"error": str(e)}
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return {"error": str(e)}


# Test with demo data (IBM stock)
stock_data = get_stock_data("IBM", api_key="YOUR_REAL_KEY_HERE") # Make sure you've replaced "YOUR_REAL_KEY_HERE" with your actual key
print("\n--- API Response Keys ---")
print(stock_data.keys())
print("-------------------------\n")


--- API Response Keys ---
dict_keys(['Meta Data', 'Time Series (Daily)'])
-------------------------



Alpha Vantage keeps that key just for quick tests, and it only gives you a small sample (IBM daily data) or sometimes just the “Information” notice you’re seeing.

What to do

Sign up (free)
Go to https://www.alphavantage.co/support/#api-key
 and enter your email.
They immediately email you a personal API key—no credit card, no waiting.

Use your key in the code
Replace the default "demo" with the key you receive:
```python
stock_data = get_stock_data("IBM", api_key="YOUR_REAL_KEY_HERE")
print(stock_data.keys())
```

or edit the function call everywhere you need it.

Respect their limits
* The free tier allows 5 calls per minute and 500 calls per day.
* If you need more, they have paid plans.

Once you use your own key, the "Information" message will be replaced with real stock data in the usual "Meta Data" and "Time Series (Daily)" fields.

## Step 3: Ask AI to Help Parse the Data

Can you think of a prompt to the AI to help write the function?  The API returns JSON. Let's ask AI how to convert it to a pandas DataFrame:

In [16]:
# Assuming stock_data is already fetched from the previous cell

def parse_stock_data(stock_data, symbol):
    """Convert API response to pandas DataFrame"""

    # Check if we got valid data
    if "Time Series (Daily)" not in stock_data:
        print("Error: No stock data found in the API response.")
        print("Available keys in response:", list(stock_data.keys()))
        return None

    # Extract the time series data
    time_series = stock_data["Time Series (Daily)"]

    # Convert to DataFrame
    df = pd.DataFrame.from_dict(time_series, orient='index')

    # Clean up column names
    df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']

    # Convert to numeric
    df = df.astype(float)

    # Convert index to datetime
    df.index = pd.to_datetime(df.index)

    # Sort by date
    df = df.sort_index()

    # Add symbol column
    df['Symbol'] = symbol

    return df

# Parse the stock data
df = parse_stock_data(stock_data, "IBM")

if df is not None:
    print("\n--- Parsed DataFrame Head ---")
    print(df.head())
    print(f"\nData shape: {df.shape}")
    print("---------------------------\n")
else:
    print("Could not parse stock data - df is None.")


--- Parsed DataFrame Head ---
              Open     High     Low   Close     Volume Symbol
2025-05-28  263.16  265.000  259.94  260.24  2318437.0    IBM
2025-05-29  260.75  261.130  256.77  258.69  2295228.0    IBM
2025-05-30  258.75  260.120  257.10  259.06  9668923.0    IBM
2025-06-02  257.85  263.976  257.22  263.90  2831881.0    IBM
2025-06-03  263.35  265.560  262.58  265.20  2494922.0    IBM

Data shape: (100, 6)
---------------------------



## Step 4: Ask AI to Help Create a Simple Chart

Try this prompt (can you make it better?):

```
How do I create a simple line chart of stock prices using pandas?
```

You might get something like.  Now let's visualize the data:

In [14]:
import matplotlib.pyplot as plt
import pandas as pd # Ensure pandas is imported if not already

stock_data = get_stock_data("IBM", api_key="YOUR_REAL_KEY_HERE")
print(stock_data.keys())
def plot_stock_price(df, symbol):
    if df is None or df.empty:
        print("No data to plot")
        return

    # Create a figure and an axes object
    fig, ax = plt.subplots(figsize=(12, 6)) # You can adjust figure size here

    # Plot the 'Close' price
    ax.plot(df.index, df['Close'], label='Close Price')

    # Set title and labels
    ax.set_title(f"{symbol} Stock Price")
    ax.set_xlabel("Date")
    ax.set_ylabel("Price")

    # Add a legend
    ax.legend()

    # Rotate x-axis labels for better readability
    plt.xticks(rotation=45)

    # Improve layout
    plt.tight_layout()

    # Show the plot
    plt.show()


    # Basic stats
    print(f"\n{symbol} Stats")
    print("Latest:", df['Close'].iloc[-1])
    print("High:",   df['Close'].max())
    print("Low:",    df['Close'].min())

dict_keys(['Meta Data', 'Time Series (Daily)'])


## Step 5: Extension Ideas

Let's expand to:
* compare multiple stocks
* build a simple analysis function (see earlier notebook)
* Real-time price updates
* More technical indicators (RSI, MACD)
* Portfolio tracking
* Price alerts
* Integration with a web interface
* Fundamental data (P/E ratios, etc.)
* News sentiment analysis



## Your Turn: Add a Feature

Try adding one feature with AI help:

In [18]:
import pandas as pd

# Ask AI to help you add a specific feature
# Example: "Help me add a function to calculate the RSI (Relative Strength Index)"

def calculate_rsi(df, window=14):
    """Calculate RSI based on AI suggestions"""
    if df is None or df.empty:
        print("No data to calculate RSI")
        return None

    # Calculate price changes
    delta = df['Close'].diff()

    # Separate gains and losses
    gain = delta.copy()
    loss = delta.copy()

    gain[gain < 0] = 0
    loss[loss > 0] = 0

    # Calculate average gain and average loss
    avg_gain = gain.rolling(window=window).mean()
    avg_loss = loss.abs().rolling(window=window).mean()

    # Calculate Relative Strength (RS)
    rs = avg_gain / avg_loss

    # Calculate Relative Strength Index (RSI)
    rsi = 100 - (100 / (1 + rs))

    # Add RSI as a new column to the DataFrame
    df['RSI'] = rsi

    return df

# Or ask for help with alerts
def price_alert_system(symbol, target_price, current_price):
    """Simple price alert system"""

    print("Price alert system - ask AI for implementation")
    return None

print("Choose a feature to add and ask Colab AI for help implementing it!")

# Example usage (assuming you have a DataFrame 'df' from previous steps)
# After running the cells to get and parse stock data into 'df', you can call:
df_with_rsi = calculate_rsi(df.copy()) # Use .copy() to avoid modifying the original df
if df_with_rsi is not None:
    print("\n--- DataFrame with RSI Head ---")
    print(df_with_rsi.head())
    print("-------------------------------\n")

Choose a feature to add and ask Colab AI for help implementing it!

--- DataFrame with RSI Head ---
              Open     High     Low   Close     Volume Symbol  RSI
2025-05-28  263.16  265.000  259.94  260.24  2318437.0    IBM  NaN
2025-05-29  260.75  261.130  256.77  258.69  2295228.0    IBM  NaN
2025-05-30  258.75  260.120  257.10  259.06  9668923.0    IBM  NaN
2025-06-02  257.85  263.976  257.22  263.90  2831881.0    IBM  NaN
2025-06-03  263.35  265.560  262.58  265.20  2494922.0    IBM  NaN
-------------------------------



## Key Takeaways

You've built a stock analysis tool that:
- Fetches real stock data from an API
- Converts JSON to pandas DataFrames
- Creates visualizations of stock prices
- Calculates basic financial metrics
- Compares multiple stocks

This demonstrates core skills for financial applications:
- API integration
- Data processing with pandas
- Basic financial analysis
- Data visualization

## Next Steps

- Get a real Alpha Vantage API key for more data
- Add more technical indicators
- Combine with the chatbot tutorial
- Build a Gradio interface for user interaction
- Integrate with portfolio tracking

The foundation you've built here can be extended into sophisticated financial analysis tools!